In [16]:
# Tổng quan bước này ta lấy dữ liệu từ mt5
# Sau đó lưu vào biến tên là data_forex
# Và nhớ khai báo các cặp tiền tệ cần thiết vào biến symbols

import requests
import pandas as pd
from datetime import datetime

"""Cảnh báo không được thay đổi địa chỉ ip của hàm này"""
def get_history(symbol: str, timeframe: str = "D1", start: str = "2014-01-01", end: str = datetime.now().date().isoformat()):
    params = {
        "timeframe": timeframe,
        "start": start,
        "end": end,
    }
    response = requests.get(f"http://221.132.33.180:8005/history/{symbol}", params=params)
    response.raise_for_status()
    data = response.json()
    data = pd.DataFrame(data)
    data["Date"] = pd.to_datetime(data['Date'], unit='s')
    data.set_index("Date", inplace=True)
    return data

# Phía trên là hàm lấy dữ liệu từ mt5  và đầu vào là cạp tiền tệ của forex, crypto, stock
# hàm trả về data của symbols đó

# symbols = {'all': ["EURCHF", "EURNZD", "NZDUSD", "AUDNZD", "USDJPY", "NZDJPY", "GBPJPY", "USDCHF"]}
symbols = {'all': ["EURCHF"]}
"""Đoạn code này có thể chỉnh sửa lấy loại tiền tệ theo yêu cầu"""
def get_data(symbols):
  data_forex = {}
  for sym in symbols["all"]:
    data_forex[sym] = get_history(sym)
    print(sym)
  return data_forex

data_forex = get_data(symbols)

EURCHF


# Bước lưu lại dữ liệu và load dữ liệu lên nếu cần

# Code này là chia sẽ cho mọi người sử dụng chung code để có thể làm việc với forex và sau đó hiểu được code

# Điều quan trọng khi làm trong code là tinh gọn code, code chạy auto, chạy theo tùng hàm và cuối cùng là đầu vào đơn giản của từng hàm.




- Các bước sắp xếp code trong dữ liệu:
1. Khai báo những thư viên công cụ được sử dụng. Lấy dữ liệu từ mt5.
2. Các bước xử lý dữ liệu gồm:  Làm sạch, lọc nhiễu, smooth, loại bỏ outlier.
3. Khi có dữ liệu ta bắt đầu xem các model được sử dụng: Bước Tuning hyperparameter, bước thứ hai là train model.
4. Bước thứ 4 là đưa ra dự đoán, xem lợi nhuận của từng đồng tiền.
5. Lưu lại tất cả những gì đã làm từ model, từ paramete,.. và các thứ khác có thể add vào model


In [3]:
!pip install finta
!pip install simdkalman
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 7.6 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import pickle

from datetime import datetime
from collections import Counter
from finta import TA
from scipy.signal import savgol_filter
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tqdm import tqdm
import os
from pathlib import Path
import glob
# thư viện làm trơn nhãn
import simdkalman
from sklearn.ensemble import IsolationForest
import plotly.express as px
# thư viện tuning
from sklearn.neural_network import MLPClassifier

# dùng torch làm cái mlp là 1 model phân lớp
import torch
from torch import nn
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from skorch.callbacks import Checkpoint
# dùng xgboost làm 1 model phân lớp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from tqdm import notebook
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Công đoạn tiếp theo là ta sẽ xử lý dữ liệu

# Công đoạn này thêm các technical indicators vào

In [22]:
# Hàm khởi tạo các  technical indicator
def create_indicators(ohlcv: pd.DataFrame) -> pd.DataFrame:
    data = ohlcv.copy()
    indi, signal = bias(ohlcv)
    data = pd.concat([data, indi], axis=1)
    data['BIAS_signal'] = signal
    data['VR'], data["VR_signal"] = vr(ohlcv)
    data['TRIX'], data["TRIX_signal"] = trix(ohlcv)
    data['ER'] = TA.ER(ohlcv)
    data['EVWMA'] = TA.EVWMA(ohlcv)
    data['VWAP'] = TA.VWAP(ohlcv)
    data['MOM'] = TA.MOM(ohlcv)
    data['ROC'] = TA.ROC(ohlcv)
    data['RSI'] = TA.RSI(ohlcv)
    data['IFT_RSI'] = TA.IFT_RSI(ohlcv)
    data['ATR'] = TA.ATR(ohlcv)
    data['BBWIDTH'] = TA.BBWIDTH(ohlcv)
    data['ADX'] = TA.ADX(ohlcv)
    data['STOCH'] = TA.STOCH(ohlcv)
    data['STOCHD'] = TA.STOCHD(ohlcv)
    data['AO'] = TA.AO(ohlcv)
    data['MI'] = TA.MI(ohlcv)
    data['MFI'] = TA.MFI(ohlcv)
    data['PZO'] = TA.PZO(ohlcv)
    data['EFI'] = TA.EFI(ohlcv)
    data['EMV'] = TA.EMV(ohlcv)
    data['CCI'] = TA.CCI(ohlcv)
    data['FISH'] = TA.FISH(ohlcv)
    data['FVE'] = TA.FVE(ohlcv)

    macd = TA.MACD(ohlcv)
    data['MACDCal'] = macd['MACD'] - macd['SIGNAL']

    macdev = TA.EV_MACD(ohlcv)
    data['EVMACD'] = macdev["MACD"]

    data['TR'] = TA.TR(ohlcv)

    DMI = TA.DMI(ohlcv)
    data['DMI+'] = DMI["DI+"]
    data['DMI-'] = DMI["DI-"]

    VORTEX = TA.VORTEX(ohlcv)
    data['VIp'] = VORTEX["VIp"]
    data['ADL'] = TA.ADL(data)

    TSI = TA.TSI(ohlcv)
    data['TSI'] = TSI["TSI"]
    data['TSIsignal'] = TSI["signal"]

    KST = TA.KST(ohlcv)
    data['KST'] = KST["KST"]

    data['CHAIKIN'] = TA.CHAIKIN(ohlcv)
    data['OBV'] = TA.OBV(ohlcv)
    data['WOBV'] = TA.WOBV(ohlcv)

    EBBP = TA.EBBP(ohlcv)
    data['EBBPBull'] = EBBP["Bull."]
    data['EBBPBear'] = EBBP["Bear."]

    BASPN = TA.BASPN(ohlcv)
    data['BASPNBuy'] = BASPN["Buy."]
    data['BASPNSell'] = BASPN["Sell."]
    data['COPP'] = TA.COPP(ohlcv)

    BASP = TA.BASP(ohlcv)
    data['BASPBuy'] = BASP["Buy."]
    data['BASPSell'] = BASP["Sell."]

    WTO = TA.WTO(ohlcv)
    data['WTOWT1'] = WTO["WT1."]

    data['STC'] = TA.STC(ohlcv)
    data['VPT'] = TA.VPT(ohlcv)

    # Có 50 kỹ thuật technical indicators ở đây
    return data

LOOK_BACK = 10
"""Hàm này dùng để phân chia data ra theo từng ngày nối đuôi nhau vựa trên lock_back"""
# Xem xem trung bình 10 ngày tăng hay giảm, là dùng để xem xu hướng của tiền

def create_dataset(data, label, look_back=1):  #
    X_ = []
    y_ = []
    # Tạo vòng lạp để lấy các lookback
    for i in range(len(data)-look_back-1):
        X_.append(data[i:(i+look_back)])
        y_.append(label[i + look_back])
    return np.array(X_), np.array(y_)

""" Hàm này để lấy các ngày liên tiếp theo n_days để làm các cột dữ liệu"""

def add_past_days_as_feature(data: pd.DataFrame, n_days: int = 5):
    data = pd.concat([data.shift(i).add_suffix(f"_{i}") for i in range(n_days)], axis=1)
    return data

" Hàm Bias dùng để so sánh giữ hai hai tính hiệu dài hạn và ngắn hạn vựa trên short_val và long_val "
def bias(prices):
    short_avg = prices['Close'].rolling(3, min_periods=1).mean()
    long_avg = prices['Close'].rolling(5, min_periods=1).mean()

    short_val = pd.Series(((prices['Close'] - short_avg) / short_avg) * 100, name="BIAS_short", index=prices.index)
    long_val = pd.Series(((prices['Close'] - long_avg) / long_avg) * 100, name="BIAS_long", index=prices.index)
    indi = pd.concat([short_val, long_val], axis=1)

    # So sánh xem dài hạn hay ngắn hạn cái nào sẽ lời hơn
    signal = pd.Series((long_val > short_val).astype(int), name="BIAS_signal", index=prices.index)
    return indi, signal

def vr(prices):
    maximum = (prices['High'] + prices['Close'].shift(1).bfill()).mean() # Lấy giá cao nhất ngày hiện tạo cộng cho ngày đóng cửa của tương lai,
    # Nếu tương lai là Nan thì cộng cho giá đóng cưa hiện tại
    minimum = (prices['Low'] + prices['Close'].shift(1).bfill()).mean()
    high = prices['High'].rolling(14, min_periods=1).mean()
    low = prices['Low'].rolling(14, min_periods=1).mean()

    # Tính chỉ số
    indi = pd.Series((maximum - minimum) / (high - low), name="VR", index=prices.index)
    signal = pd.Series((indi > 0.5).astype(int), name="VR_signal", index=prices.index)
    return indi, signal

# Hàm tính giá trị Trix
def trix(prices):
    indi = TA.TRIX(prices, 10)
    signal = pd.Series((indi < 0).astype(int), name="TRIX_signal", index=prices.index)
    return indi, signal

# Dưới đây là cáC HÀM DÙNG CHO VIỆC STACK MODEL VÀ LOẠI BỎ CÁC OUTLIER DÙNG CÔNG THỨC IQR

In [23]:

# hàm này để đánh giá mô hình tổng
# Là khi chúng ta train ra hết 3 model chúng ta xem rằng chúng kết hợp với nhau thì kết quả cuối cùng độ chính xác là bao nhiêu
def CV_ensemble(ensemble_name, ensemble_func, estimators, X_train, y_train, n_folds=5, shuffle=True, random_state=2022):
  kf = KFold(n_splits=5, random_state=random_state, shuffle=True) # Tạo số lượng dữ liệu bằng Kfold sau đó shuffle

  res_list = [] # tạo res_list lưu thông tin của từng lần chia kfold
  # Chạy tập dữ liệu Kfold
  for train_idx, valid_idx in notebook.tqdm(kf.split(X_train), total=kf.get_n_splits(), desc='Eval_CV'): # Chạy theo số lần n_splits
    X_train_train, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_train, y_valid = y_train[train_idx], y_train[valid_idx]
    # Tổng hợp kết quả test đối vơi từng cặp k riêng
    ensemble_pred_proba = ensemble_func(estimators, X_train_train, y_train_train, X_valid)


    # Lấy các đánh giá  của model với tập train test k này
    neg_log_loss = np.negative(log_loss(y_valid, ensemble_pred_proba))
    accuracy = accuracy_score(y_valid, ensemble_pred_proba.argmax(axis=1))

    res_list.append([ensemble_name, neg_log_loss, accuracy]) # Lưu kết quả lại`
  res_df = pd.DataFrame(np.vstack((res_list)))
  res_df.columns = ['model', 'log_loss', 'accuracy']  # thêm các thông số để đánh giá
  return res_df.reset_index(drop=True)


# hàm này để dự đoán bằng cách tổng hợp 3 model có train lại
def ensemble_average(estimators, X_train, y_train, X_test):
  # Hàm này vựa trên việc lấy xác suất của các model vote cái nào có xác suất cao nhất thì chọn
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  for iter in range(num_estimators):
    y_train = np.array(y_train, dtype=np.int64)
    estimators[iter].fit(X_train, y_train)
    preds.append(estimators[iter].predict_proba(X_test))


  preds_stack = np.hstack((preds))
  preds_mean = []
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class)
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1))

  avg_pred = np.vstack((preds_mean)).transpose()
  return avg_pred


# hàm này đưa ra dự đoán mà ko cần train lại
def ensemble_average_model(estimators, X_test, y_train): # esitamtors là các model và X_test là dữ liệu train
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  # đoạn này lấy tổng dữ đoán của các model trong estimators
  for iter in range(num_estimators):

    preds.append(estimators[iter].predict_proba(X_test))

  preds_stack = np.hstack((preds))
  preds_mean = []
  # Dự đoán trung bình cho mỗi nhãn
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class) # Tính cho số cột cho nhãn được dự đoán
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1)) # Tính giá trị trung bình cho dự đoán

  avg_pred = np.vstack((preds_mean)).transpose() # Lấy hết các giá trị tring bình sau đó chuyển vị
  return avg_pred


# hàm tạo ngưỡng nhiễu
def outlier_threshold(normality, k=1.5):
  q1 = np.quantile(normality, 0.2)
  q3 = np.quantile(normality, 0.8)
  threshold = q1 - k*(q3-q1) # Công thưc tạo ngưỡng nhiễu
  return threshold

# Khởi tạo và xử lý dữ liệu

In [24]:

def create_data(symbols, data_forex):
  price = {}
  # Lấy data của giá đóng cửa của loại đồng tiền
  for symbol in data_forex.keys():
      price[symbol] = data_forex[symbol]['Close'].copy()

  y = {}
  for symbol in data_forex.keys():

      y[symbol] = data_forex[symbol]["Close"].ffill()

  # x là features
  X = {}
  # Đoạn này lấy dữ liệu X là data dùng để tách train, test
  for symbol in data_forex.keys():
      X_base_features = create_indicators(data_forex[symbol])
      X[symbol] = add_past_days_as_feature(data=X_base_features, n_days=LOOK_BACK)
      X[symbol]["label"] = y[symbol].copy()
      X[symbol] = X[symbol].dropna(axis=0)
      y[symbol] = X[symbol]["label"].copy()

  return X, y

X, y  = create_data(symbols, data_forex)
    # print(symbol, X[symbol].shape)

/usr/local/lib/python3.10/dist-packages/finta/finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
/usr/local/lib/python3.10/dist-packages/finta/finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
/usr/local/lib/python3.10/dist-packages/finta/finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):


In [25]:
from sklearn.model_selection import train_test_split
config = {
    'data_name': '3d_forex',
    'random_state': 2023
}
# Hàm này tách dữ liệu và sau đó chuẩn hóa riêng cho từng mã
def split_data_scale_2(X: dict, y: dict, symbols: dict):
    data_all_money = {}
    for symbol in symbols["all"]:
        # Lấy dữ liệu gốc khi (chưa scale)
        data_original = X[symbol].copy()
        y_ = pd.Series(y[symbol])
        # Khởi tạo biến để lưu dữ liệu train và test cho từng fold của từng loại tề
        data_train_test = []

        # Chia dữ liệu thành tập huấn luyện và tập kiểm tra theo tỷ lệ 90/10
        X_train, X_test, y_train, y_test = train_test_split(data_original, y_, test_size=0.1,shuffle =False, random_state=config['random_state'])
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.fit_transform(X_test)

        data_all_money[symbol] = [X_train_scaled, X_test_scaled, y_train, y_test]
    return data_all_money # Dữ liệu được lưu ở dạng dict và 1 loại đồng tiền chưa 4 dữ liệu train và test của X và y

# data_all_money = split_data_scale_2(X, y, symbols) # Lấy dữ liệu đã scale ra



#  Ta đi smooth data để smooth ta cần biết các dữ liệu nhiễu thường xuyên, và bộ dữ liệu thực tế
# Việc cần ở đây là chỉ số nhiễu tượng trưng cho dữ liệu đó chỉ cần chỉnh bao nhiêu phần trăm bị nhiễu
def Smoothing_data(y, symbols) -> dict:
  symbolsmooths = {}
  # cho nhiều bộ smooths y khác nhau để xem cái nào tốt nhất
  for syms in symbols["all"]:
    smooths = []
    for isf in range(4,5):
        for ins in range(4,5):
            smoothing_factor = isf
            n_seasons = ins


            # --- define state transition matrix A tạo ma trận n_season+1
            state_transition = np.zeros((n_seasons+1, n_seasons+1))
            # hidden level         Đặt mặc định số đầu tiên là 1
            state_transition[0,0] = 1
            # season cycle         Sau đó ta chỉnh các thông sô khác cho mặc định dòng 1 có n_season = -1
            state_transition[1,1:-1] = [-1.0] * (n_seasons-1)

            # Tạo đường chéo chính có giá trị là 1
            state_transition[2:,1:-1] = np.eye(n_seasons-1)

            # --- observation model H
            # observation is hidden level + weekly seasonal compoenent
            observation_model = [[1,1] + [0]*(n_seasons-1)]

            # --- noise models, parametrized by the smoothing factor
            level_noise = 0.2 / smoothing_factor
            observation_noise = 0.2
            season_noise = 1e-3

            process_noise_cov = np.diag([level_noise, season_noise] + [0]*(n_seasons-1))**2
            observation_noise_cov = observation_noise**2
            # Sử dụng Kalman filter
            kf = simdkalman.KalmanFilter(
                state_transition, # Hiển thị cách model chuyển tiếp để thay đổi ( Đây là cách nhìn dữ liệu bị nhiễu)
                process_noise_cov, # ma trận biểu thị mức độ nhiễu của mô hình (Đây là để biết nhiễu thường xảy ra với tuần suât nào)
                observation_model, #  Là mô hình quan sát của hệ thông thường là ma trận hay hàm số biểu thị (Đây là  kết quả hiện tại đang cần chỉnh nhiễu)
                observation_noise_cov #  Ma trận biểu thị mức độ nhiễu trong đo lường  (Đây giống như là sai số để xem kết quả dự đoán với thực tế thì sau bnhieu)
                )
            # Sau đó ta tính toán lấy ra được các dự đoán nhiễu và trã về dự đoán bị nhiễu được quy định bằng các tham số trên
            block = y[syms]
            n_train = block.shape[0]
            n_test = 60
            result = kf.compute(block, n_test)
            predictproba = result.smoothed.states.mean[:,0]
            y_label = []
            for ivalue in range(1,len(predictproba)):
                if(predictproba[ivalue] > predictproba[ivalue-1]):
                    y_label.append(1)
                else:
                    y_label.append(-1)

            smooths.append(y_label)
    symbolsmooths[syms] = smooths
  return symbolsmooths

# symbolsmooths_y = Smoothing_data(y,symbols)


def smooth_label(y_train_ ,symbolsmooths, syms ):
  # Hàm chỉnh dữ liệu lại cho smooth
  y_train = []
  y_test = []
  # chạy từng mã gán lại nhãn đã smooth, -1 là 0 giảm, 1 là 1 tăng
  for idx in range(len(y_train_)):
      if symbolsmooths[syms][0][idx]  == -1:
          y_train.append(0)
      else:
          y_train.append(1)
  # y_test cũng vậy, phần test sẽ là phần còn lại tính từ n_split
  for idx in range(len(symbolsmooths[syms][0]) - len(y_train_)):
      if symbolsmooths[syms][0][idx+ len(y_train_)] == -1:
          y_test.append(0)
      else:
          y_test.append(1)
  y_test.append(1)
  return y_train, y_test


def create_train_test_all_data_forex(X, y, symbols):
  data_all = {}
  data_all_money = split_data_scale_2(X,y, symbols)
  symbolsmooth_y = Smoothing_data(y, symbols)

  for sym in symbols["all"]:
    # Lấy các cột của dữ liệu
    df = pd.DataFrame(X[sym] )
    col_df = df.columns

    # Lấy đúng loại tiền cần dùng
    X_train, X_test, y_train, y_test = data_all_money[sym]
    # Sau khi smooth data ta có y_train và y_test mới
    y_train, y_test = smooth_label( y_train, symbolsmooth_y, sym)

    # Sau đó  tạo ra các dataframe mới
    X_train = pd.DataFrame(X_train, columns = col_df)
    X_test =  pd.DataFrame(X_test, columns = col_df)
    # Drop các cột không cần thiết là label đi
    X_train = X_train.drop(columns = ["label"])
    X_test = X_test.drop(columns = ["label"])
    # Lấy dữ liệu tiền tệ đó ra
    data_set = pd.get_dummies(X_train, drop_first=False)
    data_all[sym] = [data_set, X_test, y_train, y_test]
  return data_all

data_all = create_train_test_all_data_forex(X, y, symbols)


In [9]:
len(data_all["EURCHF"][2])

2251

# Tuning hyperparameter cho các model và loại bỏ outlier trong dữ liệu:
1. Random forest
2. XGBoost
3. MLP

In [26]:
"""Code dưới đây loại bỏ các outlier khi vào việc train dữ liệu"""

from skorch.net import chain
# Hàm khởi tạo Isolation forest cho việc loại bỏ các outlier vựa trên decision Tree
# Nó chỉ loại bỏ 1 phần bị outliers
def create_clf(X_train):
  clf = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1,
    random_state=config['random_state'])

  clf.fit(X_train)
  return clf


# Hàm loại bỏ outlier cho tập data train
def delete_outlier(clf, X_train, y_train):
  # Lấy ngưỡng cho dữ liệu
  normality_df = pd.DataFrame(clf.decision_function(X_train), columns=['normality'])
  threshold = outlier_threshold(normality_df['normality'].values, k=1.5)

  # Sau đo ta loại bỏ dòng bị outlier
  X_train = X_train[normality_df['normality'].values>=threshold]
  y_train = y_train[normality_df['normality'].values>=threshold]
  return X_train, y_train

# hàm này tạo ra dataframe chưa bộ thông số tốt nhất cho từng model
def create_best_cv(model_name):

  model_list = []
  # mỗi model chuẩn bị 5 cái loss tốt nhất vì chạy CV
  for name in model_name:
    model_list.append(np.full(5, name))

  best_cv_df = pd.DataFrame({'model': np.hstack((model_list)), 'accuracy':None, 'best_hyper_param':None})
  return best_cv_df

def data_np(data_all, symbols):
  for sym in symbols["all"]:
    # Thay đổi dữ liệu về dạng làm tròn và numpy

    data_all[sym][2] = np.array(data_all[sym][2])
    data_all[sym][3] = np.array(data_all[sym][3])

    data_all[sym][0] = data_all[sym][0].astype(np.float32)
    data_all[sym][1] = data_all[sym][1].astype(np.float32)


# Tạo model loại bỏ nhiễu

# Tối ưu model
# Làm sao cho model chạy hiệu quả nhất và là no phải có lời

def delete_and_create_best(data_all, symbols):
  data_np(data_all, symbols)
  best_cv_df = {}

  for sym in symbols["all"]:
    # Tạo clf cho delete outlier
    clf = create_clf(data_all[sym][0])
    # xóa các outliers
    print(sym)
    data_all[sym][0], data_all[sym][2] = delete_outlier(clf, data_all[sym][0], data_all[sym][2] )
    # Tạo ra best cv cho từng mã tiền
    model_name = [ 'rf', 'xgb', 'mlp']
    best_cv_df[sym] = create_best_cv(model_name)

  return best_cv_df



In [ ]:
best_cv_df = delete_and_create_best(data_all, symbols)

EURCHF


# Đây chưa các hàm check cho model

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_accuracy(y_true, y_pred):
    """
    Tính độ chính xác giữa dự đoán và nhãn thực tế.

    Parameters:
    - y_true: Mảng chứa nhãn thực tế.
    - y_pred: Mảng chứa dự đoán của mô hình.

    Returns:
    - accuracy: Độ chính xác.
    """
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy




# Bên dưới khai bó tất cả các thư viện có model nhất


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, PassiveAggressiveClassifier


# QUy trình đầu tiên ta sẽ lấy độ chính xác của model cơ sở tốt nhất
# Sau đó ta lấy train test cho model mới
# Tiếp theo ta làm công việc chạy predict cho model mới khi stack lại với nhau
# Cuối cùng ta so sánh accuracy của nó với model mới và trả về True False theo độ chính xác

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, PassiveAggressiveClassifier

# def all_model_sklearn(X_train, y_train):
#     # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
#     X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#     # Danh sách các mô hình phân loại
#     classifiers = {
#         'KNeighborsClassifier': KNeighborsClassifier(),
#         'LogisticRegression': LogisticRegression(),
#         'SGDClassifier': SGDClassifier(),
#         'SVC': SVC(),
#         'DecisionTreeClassifier': DecisionTreeClassifier(),
#         'ExtraTreeClassifier': ExtraTreeClassifier(),
#         'GaussianNB': GaussianNB(),
#         'MultinomialNB': MultinomialNB(),
#         'ComplementNB': ComplementNB(),
#         'BernoulliNB': BernoulliNB(),

#         'AdaBoostClassifier': AdaBoostClassifier(),
#         'GradientBoostingClassifier': GradientBoostingClassifier(),
#         'BaggingClassifier': BaggingClassifier(),
#         'VotingClassifier': VotingClassifier(estimators=[('svc', SVC()), ('dt', DecisionTreeClassifier()), ('rf', RandomForestClassifier())]),
#         'StackingClassifier': StackingClassifier(estimators=[('svc', SVC()), ('dt', DecisionTreeClassifier()), ('rf', RandomForestClassifier())]),
#         'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),

#         'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
#         'MLPClassifier': MLPClassifier(),
#         'GaussianProcessClassifier': GaussianProcessClassifier(),
#         'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(),
#         'PassiveAggressiveClassifier': PassiveAggressiveClassifier(),
#         'Perceptron': Perceptron(),
#     }

#     # Huấn luyện và đánh giá từng mô hình
#     for name, model in classifiers.items():
#         model.fit(X_train, y_train)

#     return classifiers

# # Gọi hàm với dữ liệu X_train và y_train của bạn
# for sym in symbols["all"]:
#     X_train = data_all[sym][0]
#     y_train = data_all[sym][2]

#     all_classifier_model = all_model_sklearn(X_train, y_train)


In [27]:

X_train, X_test, y_train, y_test = data_all["EURCHF"]
base_classifier =  DecisionTreeClassifier(max_depth =1)
adaboost = AdaBoostClassifier(base_classifier, n_estimators = 50, random_state = 2023)
# Train model adaboost

adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

acc =  accuracy_score(y_test, y_pred)
print("Độ chính xác của model adaboost là: ", acc)

Độ chính xác của model adaboost là:  0.6374501992031872


In [ ]:
# def tune_and_evaluate(X_train, y_train, model, param_grid):
#     X_train, y_train

#     grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
#     grid_search.fit(X_train, y_train)

#     best_model = grid_search.best_estimator_
#     best_params = grid_search.best_params_


#     return best_model, best_params, accuracy

# # Danh sách các mô hình cùng với các tham số cần tinh chỉnh
# models = {
#     'SVC': (SVC(), {'C': [1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}),
#     'LogisticRegression': (LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1, 10, 100]}),
#     'KNeighborsClassifier': (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}),
#     'DecisionTreeClassifier': (DecisionTreeClassifier(), {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30]}),
#     'GaussianNB': (GaussianNB(), {}),
#     'AdaBoostClassifier': (AdaBoostClassifier(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}),
#     'GradientBoostingClassifier': (GradientBoostingClassifier(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}),
# }

# # Gọi hàm tuning và đánh giá cho từng mô hình
# for model_name, (model, param_grid) in models.items():
#     best_model, best_params, accuracy = tune_and_evaluate(X_train, y_train, model, param_grid)
#     print(f"Results for {model_name}:\nBest Params: {best_params}\nAccuracy: {accuracy}\n")

In [ ]:
# buy, sell và hold
# Dùng ảnh 1 giờ nhìn về 4 giờ tương lai tăng 1 ngưỡng nhất định thì cho là buy thường 700 điểm -700 cho là sell.
# Cái cách dùng 0.

# Muốn làm thế này phả cs model giữ lại nhưng gì cần thiết như data và model